In [13]:
#使用python进行网站性能测试
#参考https://pawelmhm.github.io/asyncio/python/aiohttp/2016/04/22/asyncio-aiohttp.html

import aiohttp
import asyncio
import nest_asyncio
import time
import random
import string
from aiohttp import ClientSession
import pandas as pd
nest_asyncio.apply()

def ranString():
    return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(8))


proxies=""
url = ""
moni_time = []
moni_status = []


async def fetch(session, url):
    b=time.time()
    try:
        async with session.get(url,proxy=proxies) as response:
            res = await response.json()
            moni_time.append((time.time()-b,response.status))
    #         moni_status.append(response.status)
            return res;
    except :
        moni_time.append((time.time()-b,0))
    
    
async def bound_fetch(sem, url, session):
    # Getter function with semaphore.
    async with sem:
        await fetch(session, url)

async def run(r):
    tasks = []
    # create instance of Semaphore
    sem = asyncio.Semaphore(100)

    # Create client session that will ensure we dont open new connection
    # per each request.
    async with ClientSession() as session:
        for i in range(r):
            # pass Semaphore and session to every GET request
            task = asyncio.ensure_future(bound_fetch(sem, url + ranString(), session))
            tasks.append(task)

        responses = asyncio.gather(*tasks)
        await responses
        
        
async def main():
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url + ranString())

moni_time = []

round_begin_time = time.time()
loop = asyncio.get_event_loop()

future = asyncio.ensure_future(run(10000))
loop.run_until_complete(future)
round_cost_time = time.time() - round_begin_time
pd.DataFrame(moni_time).describe()

ModuleNotFoundError: No module named 'aiohttp'